In [1]:
!pip install -q peft accelerate bitsandbytes
!pip install -U datasets
!pip install -U transformers
!pip install evaluate
!pip install rouge_score
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import wandb
! wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kilkabezhvosta (kilkabezhvosta-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
system_prompt = """Write a cover letter for the Job title at the company mentioned, incorporating the candidate’s relevant experiences, qualifications, and skills from their resume,
and aligning them with the responsibilities and requirements listed in the job description.
Show the candidate’s enthusiasm for working for the company mentioned by including specific reasons why the
candidate is interested in this company and how their background makes them a great fit."""

In [4]:
def preprocess_function(batch):
    # batch — словарь списков, например batch['Instruction'] — список резюме
    inputs = [system_prompt + "\nJob description:\n" + text for text in batch['Instruction']]
    targets = batch['Output']
    return {"input_text": inputs, "target_text": targets}

def tokenize_function(examples):
    inputs = tokenizer(
        examples['input_text'],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    targets = tokenizer(
        examples['target_text'],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    inputs["labels"] = targets["input_ids"]
    # Заменяем pad токены на -100
    inputs["labels"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_seq]
        for label_seq in inputs["labels"]
    ]
    return inputs



In [5]:

from datasets import load_dataset

# Подставь имя своего датасета
train_dataset = load_dataset("dhruvvaidh/cover-letter-dataset-llama3", split="train", download_mode="force_redownload")
test_dataset = load_dataset("dhruvvaidh/cover-letter-dataset-llama3", split="test", download_mode="force_redownload")
# Посмотрим пример


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/448 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/841k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/367k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/813 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/349 [00:00<?, ? examples/s]

train-00000-of-00001.parquet:   0%|          | 0.00/841k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/367k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/813 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/349 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# === 2. Добавляем паддинг токен, если нужно ===
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# === 3. Загружаем модель ===
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")

# === 4. Обновляем размер эмбеддингов (обязательно после добавления pad_token) ===
model.resize_token_embeddings(len(tokenizer))


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

Embedding(50258, 2048)

In [7]:
from datasets import load_dataset
from peft import LoraConfig, get_peft_model

# Подготовка LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Далее — подготовка датасета, Trainer и обучение


trainable params: 1,572,864 || all params: 1,313,103,872 || trainable%: 0.1198


In [8]:
train_dataset = train_dataset.map(preprocess_function, batched=True).map(tokenize_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True).map(tokenize_function, batched=True)


Map:   0%|          | 0/813 [00:00<?, ? examples/s]

Map:   0%|          | 0/813 [00:00<?, ? examples/s]

Map:   0%|          | 0/349 [00:00<?, ? examples/s]

Map:   0%|          | 0/349 [00:00<?, ? examples/s]

In [9]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./lora-finetuned",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="steps",
    eval_steps=500,
    logging_steps=100,
    save_steps=100,
    learning_rate=2e-4,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,  # если есть GPU с поддержкой
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)


/tmp/ipython-input-9-2641416619.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: kilkabezhvosta (kilkabezhvosta-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
500,4.964900,5.001309
1000,4.875400,4.928382
1500,4.771900,4.877252
2000,4.700900,4.866885


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

TrainOutput(global_step=2040, training_loss=4.910805668550379, metrics={'train_runtime': 2118.0878, 'train_samples_per_second': 3.838, 'train_steps_per_second': 0.963, 'total_flos': 3.022456847597568e+16, 'train_loss': 4.910805668550379, 'epoch': 10.0})

In [16]:
resume = """
Job Title: Data Scientist
Preferred Qualifications:
BSc focused on data Science/computer Science/engineering
4+ years experience Developing and shipping production grade machine learning systems
2+ years building and shipping data Science based personalization services and recommendation systems
experience in data Science or machine learning engineering
Strong analytical and data Science skills
Hiring Company:
XYZ Corporation
Applicant Name:
John Smith
Working Experience: Data Analyst at ABC Company
Machine Learning Engineer at DEF Company
Qualifications: BSc in Computer Science, 5+ years of experience in data science and machine learning
Skillsets:
Python, R, scikit-learn, Keras, Tensorflow
"""

prompt = system_prompt + "\nJob description:\n" + resume + "\n\nGenerate a personalized cover letter based on this."
print("PROMPT:\n", prompt)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=True,
    top_p=0.9,
    temperature=0.8,
    eos_token_id=tokenizer.eos_token_id
)

print("PREDICTION:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


PROMPT:
 Write a cover letter for the Job title at the company mentioned, incorporating the candidate’s relevant experiences, qualifications, and skills from their resume,
and aligning them with the responsibilities and requirements listed in the job description.
Show the candidate’s enthusiasm for working for the company mentioned by including specific reasons why the
candidate is interested in this company and how their background makes them a great fit.
Job description:

Job Title: Data Scientist
Preferred Qualifications:
BSc focused on data Science/computer Science/engineering
4+ years experience Developing and shipping production grade machine learning systems
2+ years building and shipping data Science based personalization services and recommendation systems
experience in data Science or machine learning engineering
Strong analytical and data Science skills
Hiring Company:
XYZ Corporation
Applicant Name:
John Smith
Working Experience: Data Analyst at ABC Company
Machine Learning